# Find, Load, and Visualise Earth Observation Imagery

This notebook demonstrates how to find, load, and visualise Earth observation imagery using cloud native approaches, which work well on your desktop or in cloud environments!

The notebook demonstrates how to find images from the Element 84 Earth Search STAC catalog, load them using `odc-stac`, and visualise them using extensions to xarray provided by `odc-geo`.

## Set up

The first step is to set up the required Python libraries and local imports.

* `odc.stac` and `pystac_client` are used to access the Earth Search STAC catalog
* `numpy` is used to manipulate data


In [ ]:
from odc.stac import configure_s3_access, load
from pystac.client import Client
import numpy as np

The second step is to start a Dask client.

Dask supports local parallel processing and can help speed up computation times.

In [ ]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

## Part 1: Find

### 1.1 Connecting to the catalog

In [ ]:
# The catalog URL for the Earth Search STAC API
catalog = "https://earth-search.aws.element84.com/v1"

# Pystac_client's Client class is used to connect to the catalog
stac_client = Client.open(catalog)

# Configure settings for reading from Earth Search STAC
configure_s3_access(
    aws_unsigned=True,
)

### 1.2 Selecting an area to query

In [ ]:
from odc.geo import BoundingBox

# Auckland, New Zealand
bbox = BoundingBox(
    left=174.65,
    bottom=-36.90,
    right=174.90,
    top=-36.75,
    crs="EPSG:4326",
)

bbox.explore()

### 1.3 Set year and month to query

In [ ]:
# Set a date (year-month) for querying
date_query = "2025-09"

### 1.4 Choose collections and filters

In [ ]:
# Set Earth Search product ID as the STAC "Collection"
collections_query = ["sentinel-2-l2a"]

# Set up a filter query. This is less than 20% cloud cover
filter_query = {"eo:cloud_cover": {"lt": 20}}

### 1.5 Running the query to indentify matching STAC items

In [ ]:
# Query with filtering for cloud cover
items = stac_client.search(
    collections=collections_query,
    intersects=bbox.polygon,
    datetime=date_query,
    query=filter_query,
).item_collection()

print(f"Found {len(items)} items within the bounding box, date range, and cloud cover filter.")

## Part 2: Load

### 2.1 Using odc-stac to load identified items

This may take a few minutes

In [ ]:
# Load our filtered data
ds_filtered = load(
    items,
    bands=["red", "green", "blue"],
    crs="utm",
    chunks={},
    resolution=10,
    groupby="solar_day",
    intersects=bbox.polygon,
).compute()

ds_filtered

### 2.2 Review loaded imagery

Identify which image you want to export and note the date.

In [ ]:
# To_array sets up a 3D array with the time dimension, which works directly
# with the plot function to make an RGB image
ds_filtered.to_array().plot.imshow(col="time", col_wrap=2, robust=True, size=5)

## Part 3: Visualise

### 3.1 Select best image

Update the `best_image_date` parameter to match the date you identified in the previous step.

In [ ]:
best_image_date = "2025-09-09"

best_image = ds_filtered.sel(time=best_image_date).squeeze()

best_image

### 3.2 View the selected image on an interactive map

In [ ]:
visualisation = best_image.odc.to_rgba()

visualisation.odc.explore()

### 3.3 Improve the image contrast

Calculate the values corresponding to the 1st and 99th percentiles. This turns the flat (low contrast) image into a useful visualisation by ignoring outliers.
These can be used in the `to_rgba()` function to stretch the image.

In [ ]:
percentile_stretch = (1, 95)

rgb_array = best_image.to_array().values

stretch_vmin, stretch_vmax = np.nanpercentile(rgb_array, percentile_stretch)

Apply the percentile stretch values to the visualisation

In [ ]:
stretch_visualisation = best_image.odc.to_rgba(vmin=stretch_vmin, vmax=stretch_vmax)

stretch_visualisation.odc.explore()

### 3.4 Export to a cloud-optimised GeoTIFF

In [ ]:
stretch_visualisation.odc.write_cog("sentinel2_example.tif", overwrite=True)

## Part 4: Tidy up

In this section, we close the Dask client.
This prevents multiple clients being instantiated when using different notebooks.

In [ ]:
dask_client.close()